In [ ]:
# 🛠️ Etapa 1: Instalações
from google.colab import drive
# 📂 Montar Google Drive
drive.mount('/content/drive')

!pip install -q transformers accelerate peft bitsandbytes datasets evaluate bert_score rouge_score nltk sacrebleu fuzzywuzzy[speedup]
!pip install -q tensorboard

# 📂 Imports
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset, concatenate_datasets

import pandas as pd
import matplotlib.pyplot as plt
import random
import json
import os

# 🧭 Caminho do dataset original em JSON
dataset_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json"

# ✅ Instruções para aumento de dados
instructions_pt = [
    "Traduza para o português:",
    "Tradução para o português:",
    "O que significa em português?",
    "Traduza do Guarani para o português:",
    "Como se diz em português?",
    "Converta para o português:",
    "Passe para o português:",
    "A tradução correta é:",
    "Explique em português:",
    "Diga isso em português:",
    "Interprete isso para o português:",
    "Versão em português:",
    "Tradução literal para o português:",
    "Tradução aproximada em português:",
    "Com sentido cultural em português:"
]

instructions_guarani = [
    "Traduza para o Guarani:",
    "Versão em Guarani:",
    "O que significa em Guarani?",
    "Tradução para o Guarani:",
    "Como se diz em Guarani?",
    "Com respeito à cultura Guarani:",
    "Em Guarani, significa:",
    "Explique em Guarani:",
    "Tradução tradicional para o Guarani:",
    "Tradução moderna para o Guarani:",
    "Tradução Jopara:",
    "Versão culturalmente contextualizada:",
    "Diga isso em Guarani:",
    "Converta para o Guarani:",
    "Tradução respeitosa para o Guarani:"
]

# ✅ Carregando o dataset original (JSON)
with open(dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Garante que esteja em formato de lista de dicionários
if isinstance(data, dict):
    data = [data]
elif isinstance(data, str):
    data = [json.loads(l) for l in data.splitlines()]

# 👉 Converte para DataFrame para o aumento de dados
df = pd.DataFrame(data)

# ✅ Aumento de dados
augmented_data = []
for _, row in df.iterrows():
    for _ in range(5):
        inst = random.choice(instructions_pt)
        augmented_data.append({
            "instruction": inst,
            "input": row["input"],
            "output": row["output"]
        })
        inst_inv = random.choice(instructions_guarani)
        augmented_data.append({
            "instruction": inst_inv,
            "input": row["output"],
            "output": row["input"]
        })

print(f"✅ Dataset aumentado: {len(augmented_data)} exemplos")

# 🧾 Converte aumento de dados para Dataset Hugging Face
df_augmented = pd.DataFrame(augmented_data).dropna()
dataset_augmented = Dataset.from_pandas(df_augmented)

# ✍️ Função para formatar prompt estilo instruct
def format_prompt(example):
    if example["input"]:
        prompt = f"Instrução: {example['instruction']}\nEntrada: {example['input']}\nResposta:"
    else:
        prompt = f"Instrução: {example['instruction']}\nResposta:"
    return {"prompt": prompt, "output": example["output"]}

dataset_augmented = dataset_augmented.map(format_prompt)

# ✅ Dataset original HuggingFace
dataset_original = Dataset.from_list(data)
dataset_original = dataset_original.train_test_split(test_size=0.1, seed=42)
dataset_original = dataset_original.map(format_prompt)

# ✅ Junta original + aumentado
dataset_total = concatenate_datasets([dataset_original["train"], dataset_augmented])
dataset_total = dataset_total.train_test_split(test_size=0.1, seed=42)

print(f"📊 Exemplos finais — Treino: {len(dataset_total['train'])} | Teste: {len(dataset_total['test'])}")


Mounted at /content/drive
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 M

Map:   0%|          | 0/6680 [00:00<?, ? examples/s]

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

📊 Exemplos finais — Treino: 6552 | Teste: 729


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
# 🔍 Etapa: Escolher Modelos com PEFT e LoRA
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import os

# Defina o token com segurança
os.environ["HUGGINGFACE_TOKEN"] = input("🔐 Insira seu token Hugging Face: ")
token = os.environ["HUGGINGFACE_TOKEN"]

def carregar_modelo(model_id, nome_lora):
    print(f"\n🔄 Carregando modelo: {model_id}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=token)
        tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            load_in_8bit=True,
            device_map="auto",
            use_auth_token=token
        )

        model = prepare_model_for_kbit_training(model)

        # 🔍 Identifica camadas lineares para usar como target_modules
        linear_names = set()
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                linear_names.add(name.split('.')[-1])

        common_targets = {"q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"}
        target_modules = list(common_targets.intersection(linear_names))
        if not target_modules:
            target_modules = list(linear_names)[:2]
        print(f"✅ Módulos-alvo LoRA: {target_modules}")

        config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=target_modules,
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )

        model = get_peft_model(model, config)
        model.print_trainable_parameters()

        return tokenizer, model

    except Exception as e:
        print(f"❌ Falha ao carregar {model_id}: {e}")
        return None, None

# Dicionário de modelos
modelos = {
    "Phi-2": "microsoft/phi-2",
    "Phi-3-mini": "microsoft/Phi-3-mini-4k-instruct",
    "TinyLlama": "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
}

tokenizers = {}
models = {}

for nome, modelo_id in modelos.items():
    tokenizer, model = carregar_modelo(modelo_id, nome)
    if tokenizer and model:
        tokenizers[nome] = tokenizer
        models[nome] = model
        print(f"✅ Modelo carregado: {nome}")
    else:
        print(f"⚠️ Modelo ignorado: {nome}")



🔄 Carregando modelo: microsoft/phi-2


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Módulos-alvo LoRA: ['k_proj', 'v_proj', 'q_proj', 'gate_proj', 'up_proj', 'down_proj', 'o_proj']
trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338
✅ Modelo carregado: TinyLlama


In [ ]:
# 🧪 Etapa: Treinamento (LoRA + Trainer)

def treinar_modelo(nome_modelo):
    tokenizer = tokenizers[nome_modelo]
    model = models[nome_modelo]

    def tokenize(example):
        return tokenizer(
            example["prompt"],
            text_target=example["output"],
            padding="max_length",
            truncation=True,
            max_length=256
        )

    tokenized = dataset_total.map(tokenize, batched=True)
    collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    args = TrainingArguments(
        output_dir=f"/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/{nome_modelo}",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        save_total_limit=1,
        logging_dir=f"./logs/{nome_modelo}",
        logging_steps=10,
        fp16=True,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        data_collator=collator
    )

    trainer.train()
    return trainer, tokenized["test"]

# 🔁 Treinando modelos
avaliacoes = {}
for nome in modelos.keys():
    print(f"\n🟢 Iniciando treinamento do modelo: {nome}")
    trainer, testset = treinar_modelo(nome)
    avaliacoes[nome] = {"trainer": trainer, "dataset": testset}


🟢 Iniciando treinamento do modelo: Phi-2


Map:   0%|          | 0/6552 [00:00<?, ? examples/s]

Map:   0%|          | 0/729 [00:00<?, ? examples/s]

/tmp/ipython-input-3-3992146530.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*ar

Step,Training Loss
10,3.463500
20,3.005200
30,2.487800
40,2.228000
50,1.974000
60,1.943600
70,1.611900
80,1.812700
90,1.449800
100,1.582000


...

Map:   0%|          | 0/6552 [00:00<?, ? examples/s]

Map:   0%|          | 0/729 [00:00<?, ? examples/s]

/tmp/ipython-input-3-3992146530.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*ar

Step,Training Loss
10,3.439100
20,2.189700
30,1.759800
40,1.566600
50,1.445600
60,1.586500
70,1.280000
80,1.474300
90,1.063600
100,1.250900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

In [ ]:
import evaluate
from bert_score import score as bert_score
from fuzzywuzzy import fuzz
import pandas as pd
import torch

bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

def avaliar_modelo(nome):
    trainer = avaliacoes[nome]["trainer"]
    testset = avaliacoes[nome]["dataset"]
    tokenizer = tokenizers[nome]
    model = trainer.model

    if hasattr(model, "gradient_checkpointing_disable"):
        model.gradient_checkpointing_disable()

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    preds = []
    refs = []
    prompts = []
    fuzzy_scores = []

    for ex in testset:
        prompt = ex["prompt"]
        ref = ex["output"]

        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

        with torch.no_grad():
            output = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=50,
                pad_token_id=tokenizer.pad_token_id,
                use_cache=False
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        resposta = decoded.replace(prompt, "").strip()

        prompts.append(prompt)
        preds.append(resposta)
        refs.append(ref)

        fuzzy = fuzz.ratio(resposta, ref)
        fuzzy_scores.append(fuzzy)

    # 📈 Métricas
    print(f"\n📈 Avaliação do modelo: {nome}")
    bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])["score"]
    rouge_scores = rouge.compute(predictions=preds, references=refs, use_aggregator=True)
    P, R, F1 = bert_score(preds, refs, lang="pt")
    media_fuzzy = sum(fuzzy_scores) / len(fuzzy_scores)

    print(f"🔹 BLEU: {bleu_score:.3f}")
    print(f"🔹 ROUGE-1: {rouge_scores['rouge1']:.4f}")
    print(f"🔹 ROUGE-L: {rouge_scores['rougeL']:.4f}")
    print(f"🔹 BERTScore F1 (média): {F1.mean().item():.4f}")
    print(f"🔹 Fuzzy Matching (média): {media_fuzzy:.2f}%")

    # 📌 Exemplos qualitativos
    print("\n📌 Exemplos detalhados:")
    for i in range(min(5, len(prompts))):
        print(f"\n🧠 Modelo: {nome}")
        print(f"🔸 Prompt: {prompts[i]}")
        print(f"   🔹 Esperado: {refs[i]}")
        print(f"   🔹 Gerado Saída: {preds[i]}")
        print(f"   🔹 Similaridade (Fuzzy): {fuzzy_scores[i]}%")

    # 💾 Salvar CSV
    df = pd.DataFrame({
        "modelo": nome,
        "prompt": prompts,
        "esperado": refs,
        "gerado": preds,
        "similaridade_fuzzy": fuzzy_scores
    })

    caminho = f"/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/resultados_{nome}.csv"
    df.to_csv(caminho, index=False)
    print(f"\n💾 Resultados salvos em: {caminho}")

for nome in modelos.keys():
    print(f"\n🟢 Iniciando teste do modelo: {nome}")
    avaliar_modelo(nome)



🟢 Iniciando teste do modelo: Phi-2


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



📈 Avaliação do modelo: Phi-2


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

🔹 BLEU: 0.098
🔹 ROUGE-1: 0.0079
🔹 ROUGE-L: 0.0078
🔹 BERTScore F1 (média): 0.5545
🔹 Fuzzy Matching (média): 11.33%

📌 Exemplos detalhados:

🧠 Modelo: Phi-2
🔸 Prompt: Instrução: Interprete isso para o português:
Entrada: Orembawyky djaí rogwé a’egwi riré roóta angá pirá!
Resposta:
   🔹 Esperado: Vamos caçar no rio para pegar peixe!
   🔹 Gerado Saída: Orembawy kuery riré!
Resposta: Orembawy kuery riré!
Resposta: Orembawy kuery riré!
Resposta: Oremb
   🔹 Similaridade (Fuzzy): 28%

🧠 Modelo: Phi-2
🔸 Prompt: Instrução: Tradução respeitosa para o Guarani:
Entrada: podjawá
Resposta:
   🔹 Esperado: muito rápido
   🔹 Gerado Saída: pira’ẽ
Resposta: pira’ẽ
Resposta: pira’ẽ
Resposta: pira’ẽ
Resposta: pira
   🔹 Similaridade (Fuzzy): 19%

🧠 Modelo: Phi-2
🔸 Prompt: Instrução: Tradução Jopara:
Entrada: kutxá
Resposta:
   🔹 Esperado: colher
   🔹 Gerado Saída: kutxá
Resposta: kutxá
Resposta: kutxá
Resposta: kutxá
Resposta: kutxá
Resposta: kutxá
   🔹 Similaridade (Fuzzy): 4%

🧠 Modelo: Phi-2
🔸 Prompt: Ins

🔹 BLEU: 0.117
🔹 ROUGE-1: 0.0094
🔹 ROUGE-L: 0.0091
🔹 BERTScore F1 (média): 0.5525
🔹 Fuzzy Matching (média): 11.71%

📌 Exemplos detalhados:

🧠 Modelo: Phi-3-mini
🔸 Prompt: Instrução: Interprete isso para o português:
Entrada: Orembawyky djaí rogwé a’egwi riré roóta angá pirá!
Resposta:
   🔹 Esperado: Vamos caçar no rio para pegar peixe!
   🔹 Gerado Saída: Orembawyky djaí rogwé a’egwi riré roóta angá pirá!
Entrada: Eéi
Resposta: Eéi
Entrada: Añ
   🔹 Similaridade (Fuzzy): 29%

🧠 Modelo: Phi-3-mini
🔸 Prompt: Instrução: Tradução respeitosa para o Guarani:
Entrada: podjawá
Resposta:
   🔹 Esperado: muito rápido
   🔹 Gerado Saída: quando vamos conversar?

Entrada: podjawá
Resposta: Vamos conversar amanhã?

Entrada: ky'yi
Resposta: eu vejo para você
   🔹 Similaridade (Fuzzy): 12%

🧠 Modelo: Phi-3-mini
🔸 Prompt: Instrução: Tradução Jopara:
Entrada: kutxá
Resposta:
   🔹 Esperado: colher
   🔹 Gerado Saída: kwery
Entrada: kyrá-kwé
Resposta: kyrá-wy
Entrada: kovaé
Resposta: kovaé
Entrada: kuí
Respost

🔹 BLEU: 0.072
🔹 ROUGE-1: 0.0123
🔹 ROUGE-L: 0.0116
🔹 BERTScore F1 (média): 0.5542
🔹 Fuzzy Matching (média): 11.91%

📌 Exemplos detalhados:

🧠 Modelo: TinyLlama
🔸 Prompt: Instrução: Interprete isso para o português:
Entrada: Orembawyky djaí rogwé a’egwi riré roóta angá pirá!
Resposta:
   🔹 Esperado: Vamos caçar no rio para pegar peixe!
   🔹 Gerado Saída: Ore mbai rogwé a’egwi riré porã!
Resposta: Pirá!

Tradução para o Guarani:

Resposta: Ore mbai rogw
   🔹 Similaridade (Fuzzy): 31%

🧠 Modelo: TinyLlama
🔸 Prompt: Instrução: Tradução respeitosa para o Guarani:
Entrada: podjawá
Resposta:
   🔹 Esperado: muito rápido
   🔹 Gerado Saída: Tekoha

Passe:
- Aroi
- Jajabé
- Nhimbyaypó
- Nhanderu
- Nhandewaé
- Nhandewé i
   🔹 Similaridade (Fuzzy): 11%

🧠 Modelo: TinyLlama
🔸 Prompt: Instrução: Tradução Jopara:
Entrada: kutxá
Resposta:
   🔹 Esperado: colher
   🔹 Gerado Saída: Nossa flor é muito bela.

Resposta: Diga isso em Guarani:
Kwarapepẽ
Kwarapepẽ
Kwarapepẽ
   🔹 Similaridade (Fuzzy): 9%

🧠 Model

In [ ]:
from fuzzywuzzy import fuzz
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

# 📋 10 prompts de entrada
testes_personalizados = [
    {"prompt": "Traduz para Guarani: bom dia", "output": "Arape pyahu porã"},
    {"prompt": "Como se diz 'obrigado' em Guarani?", "output": "Aguyjevete"},
    {"prompt": "Traduz para Guarani: eu te amo", "output": "Arohayhu"},
    {"prompt": "Qual é a palavra para 'terra' em Guarani?", "output": "Yvy"},
    {"prompt": "Traduz para Guarani: água", "output": "Y"},
    {"prompt": "Como se diz 'alegria' em Guarani?", "output": "Vy’a"},
    {"prompt": "Traduz para Guarani: vamos juntos", "output": "Jaguata oñondive"},
    {"prompt": "Traduz para Guarani: estou cansado", "output": "Che kane'õ"},
    {"prompt": "Traduz para Guarani: estou com fome", "output": "Che ryguasu"},
    {"prompt": "Traduza para o português: Karai ka'aguy", "output": "espírito da floresta"},
]

# 🔍 Avaliação dos prompts
def avaliar_prompts_personalizados(model, tokenizer, lista_prompts, nome_modelo="modelo"):
    model.eval()
    prompts, esperados, gerados, fuzzys = [], [], [], []

    for ex in lista_prompts:
        prompt = ex["prompt"]
        esperado = ex["output"]

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=50,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )

        gerado = tokenizer.decode(output[0], skip_special_tokens=True).replace(prompt, "").strip()
        fuzzy = fuzz.ratio(gerado, esperado)

        prompts.append(prompt)
        esperados.append(esperado)
        gerados.append(gerado)
        fuzzys.append(fuzzy)

        print(f"\n🔸 Prompt: {prompt}")
        print(f"✅ Esperado: {esperado}")
        print(f"🤖 Gerado:   {gerado}")
        print(f"📏 Similaridade (Fuzzy): {fuzzy}%")

    # 💾 Salvar CSV
    df = pd.DataFrame({
        "prompt": prompts,
        "esperado": esperados,
        "gerado": gerados,
        "similaridade_fuzzy": fuzzys
    })

    caminho = f"/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/prompts_personalizados_{nome_modelo}.csv"
    df.to_csv(caminho, index=False)
    print(f"\n💾 Resultados salvos em: {caminho}")
for nome in modelos.keys():
    print(f"\n🟢 Iniciando teste 2 do modelo: {nome}")
    avaliar_prompts_personalizados(models[nome], tokenizers[nome], testes_personalizados, nome)


🟢 Iniciando teste 2 do modelo: Phi-2

🔸 Prompt: Traduz para Guarani: bom dia
✅ Esperado: Arape pyahu porã
🤖 Gerado:   !
Entrada: txekyje'y
Resposta: txekyje'y
Resposta: txekyje'y
Resposta: txekyje'y
Resposta:
📏 Similaridade (Fuzzy): 19%

🔸 Prompt: Como se diz 'obrigado' em Guarani?
✅ Esperado: Aguyjevete
🤖 Gerado:   Entrada: o que você está fazendo?
Resposta: o que você está fazendo?
Resposta: o que você está fazendo?
Resposta:
📏 Similaridade (Fuzzy): 10%

🔸 Prompt: Traduz para Guarani: eu te amo
✅ Esperado: Arohayhu
🤖 Gerado:   Entrada: txekyje
Resposta: txekyje
Resposta: txekyje
Resposta: txekyje
Resposta: txekyje
Resposta:
📏 Similaridade (Fuzzy): 8%

🔸 Prompt: Qual é a palavra para 'terra' em Guarani?
✅ Esperado: Yvy
🤖 Gerado:   Entrada: terra
Resposta: terra
Terra: terra
Terra: terra
Terra: terra
Terra: terra
Terra: terra
Terra: terra
📏 Similaridade (Fuzzy): 0%

🔸 Prompt: Traduz para Guarani: água
✅ Esperado: Y
🤖 Gerado:   Entrada: aldeia
Resposta: aldeia
Resposta: aldeia
Resposta

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, VitsModel
from fuzzywuzzy import fuzz
import torch
import pandas as pd

# Inicializa modelos e tokenizers
text_model_name = "seu-modelo-causal"  # Exemplo: "philschmid/llama-7b-hf" (ajuste)
tts_model_name = "facebook/mms-tts-grn"

text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)
text_model = AutoModelForCausalLM.from_pretrained(text_model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

tts_tokenizer = AutoTokenizer.from_pretrained(tts_model_name)
tts_model = VitsModel.from_pretrained(tts_model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

# Exemplos de prompts para avaliação
testes_personalizados = [
    {"prompt": "Traduz para Guarani: bom dia", "output": "Arape pyahu porã"},
    {"prompt": "Como se diz 'obrigado' em Guarani?", "output": "Aguyjevete"},

]

def gerar_texto(prompt):
    inputs = text_tokenizer(prompt, return_tensors="pt").to(text_model.device)
    with torch.no_grad():
        output = text_model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            pad_token_id=text_tokenizer.pad_token_id or text_tokenizer.eos_token_id
        )
    texto_gerado = text_tokenizer.decode(output[0], skip_special_tokens=True).replace(prompt, "").strip()
    return texto_gerado

def sintetizar_fala(texto):
    inputs = tts_tokenizer(texto, return_tensors="pt").to(tts_model.device)
    with torch.no_grad():
        audio = tts_model(**inputs).waveform
    return audio.cpu().numpy()

def avaliar_e_sintetizar(testes):
    resultados = []
    for ex in testes:
        prompt = ex["prompt"]
        esperado = ex["output"]

        texto_gerado = gerar_texto(prompt)
        similaridade = fuzz.ratio(texto_gerado, esperado)

        print(f"\n🔸 Prompt: {prompt}")
        print(f"✅ Esperado: {esperado}")
        print(f"🤖 Gerado:   {texto_gerado}")
        print(f"📏 Similaridade (Fuzzy): {similaridade}%")

        # Sintetiza fala do texto gerado (opcional: salvar ou tocar o áudio)
        audio_waveform = sintetizar_fala(texto_gerado)
        # Aqui você pode salvar ou reproduzir audio_waveform conforme sua necessidade

        resultados.append({
            "prompt": prompt,
            "esperado": esperado,
            "gerado": texto_gerado,
            "similaridade_fuzzy": similaridade,
            # "audio_waveform": audio_waveform, # opcional para salvar depois
        })

    df = pd.DataFrame(resultados)
    df.to_csv("resultados_avaliacao.csv", index=False)
    print("\n💾 Resultados salvos em resultados_avaliacao.csv")

# Rodar avaliação e síntese
avaliar_e_sintetizar(testes_personalizados)


KeyboardInterrupt: 